# 学習モデルを使って実際のレース予想をする

In [143]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
dfs = {}

In [331]:
place = "toride"
race = "7"

### 予想したいレースのURLを以下に入力　ここのサイトから選ぶ　https://keirin.kdreams.jp/

In [332]:
url = "https://keirin.kdreams.jp/" + place + "/racedetail/23202109090200" + race.zfill(2) + "/"

In [333]:
columns = ['予想', '好気合', '総評', '枠番', '車番', '選手名府県/年齢/期別', '級班', '脚質', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外']

### データを取ってくる

In [334]:
race_data = pd.read_html(url)[4][:-1]
race_data.columns = columns
race_data = race_data.astype(str)
time.sleep(1)

### データを加工する

In [335]:
race_data = pd.concat([race_data, race_data["選手名府県/年齢/期別"].str.split("/", expand=True)], axis=1).drop("選手名府県/年齢/期別", axis=1)
race_data = race_data.drop(0, axis=1)

race_data["予想"] = pd.Categorical(race_data["予想"], categories=["nan", "×", "▲", "△", "○", "◎", "注"])
race_data["好気合"] = pd.Categorical(race_data["好気合"], categories=["★"])
race_data["脚質"] = pd.Categorical(race_data["脚質"], categories=["両", "追", "逃"])
race_data["級班"] = pd.Categorical(race_data["級班"], categories=["A1", "A2", "A3", "L1", "S1", "S2", "SS"])

prediction = pd.get_dummies(race_data["予想"])
isFine = pd.get_dummies(race_data["好気合"])
leg_type = pd.get_dummies(race_data["脚質"])
group = pd.get_dummies(race_data["級班"])

race_data = pd.concat([race_data, prediction, isFine, leg_type, group], axis=1).drop(["予想", "好気合", "脚質", "級班"], axis=1)
race_data = race_data.replace(["失", "落", "故", "欠"], 9)
race_data["ギヤ倍数"] = race_data["ギヤ倍数"].map(lambda x: x[:4] if len(x)>4 else x)
race_data[2] = race_data[2].map(lambda x: x.replace(" （欠車）", "") if "欠車"in x else x)
race_data = race_data.astype("float64")
def minmax_norm(columns):
    df = race_data[columns]
    for column in columns:
        race_data[columns] = (df - df.min()) / (df.max() - df.min())

minmax_columns = ["総評", "ギヤ倍数", "競走得点", "1着", "2着", "3着", "着外", 1, 2]
minmax_norm(minmax_columns)
race_data = race_data.loc[:, ['車番', '総評', '枠番', 'ギヤ倍数', '競走得点', '1着', '2着', '3着', '着外', 1, 2, 'nan', '×', '▲', '△', '○', '◎', '注', '★', '両', '追', '逃', 'A1', 'A2', 'A3', 'L1', 'S1', 'S2', 'SS']]

### 学習済みモデルに入れて予想する

In [336]:
import tensorflow as tf
from tensorflow import keras

In [337]:
model = tf.keras.models.load_model("pred_top3")

In [338]:
race_data.shape

(7, 29)

In [339]:
results = model.predict(race_data)
results

array([[0.29137927],
       [0.41249448],
       [0.9104875 ],
       [0.17039359],
       [0.0523212 ],
       [0.34340245],
       [0.6724162 ]], dtype=float32)

In [340]:
import numpy as np
def printResult(results):
    new_results = []
    for result in results:
        new_results.append(result[0])
        
    sorted_result = np.argsort(new_results)
    sorted_result = list(map(lambda x: x+1, sorted_result))
    new_results = sorted(new_results)
    return sorted_result[::-1], new_results[::-1]

In [341]:
results, results_pred = printResult(results)

In [342]:
df = pd.DataFrame(results_pred, results, columns=["3着以内の確率"])

In [343]:
df

,3着以内の確率
3,0.910487
7,0.672416
2,0.412494
6,0.343402
1,0.291379
4,0.170394
5,0.052321


In [344]:
dfs[place+race] = df